### Functions:

In [ ]:
def ask_llm(prompt):
  import openai
  full_response = []
  for chunk in openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[{
          "role": "user",
          "content": prompt
      }],
      stream=True,
  ):
      content = chunk["choices"][0].get("delta", {}).get("content")
      if content is not None:
          full_response.append(content)
          print(content, end='')

  return full_response

In [ ]:
def get_near_text(question, model=None):
  if model:
    embedding = model.encode(question)
    nearVector = {"vector": embedding}

  response = (
      client.query
      .get("ServicePublic", ["text", "url", "subdomain", "title", "paragraph", "line_n"])
      .with_near_vector(nearVector)
      .with_limit(10)
      .with_additional(['certainty'])
      .do()
  )

  return response

In [ ]:
def get_page(url):
  url_filter = {
    "path": ["url"],
    "operator": "Equal",
    "valueText": url,
  }

  sort_order = [
    {
      'path': ['line_n'],
      'order': 'asc'
    },
  ]

  response = (
    client.query
    .get("ServicePublic", ["text", "url"])
    .with_where(url_filter)
    .with_sort(sort_order)
    .do()
  )

  page = [result['text'] for result in response["data"]["Get"]["ServicePublic"]]

  return "\n".join(page)

In [ ]:
def get_paragraph(url, paragrah_number):
  url_filter = {
    "path": ["url"],
    "operator": "Equal",
    "valueText": url,
  }

  paragraph_filter = {
    "path": ["paragraph"],
    "operator": "Equal",
    "valueNumber": paragrah_number,
  }

  total_filter = {
      "operator": "And",
      "operands": [url_filter, paragraph_filter]
  }

  sort_order = [
    {
      'path': ['line_n'],
      'order': 'asc'
    },
  ]

  response = (
    client.query
    .get("ServicePublic", ["text", "url"])
    .with_where(total_filter)
    .with_sort(sort_order)
    .do()
  )

  page = [result['text'] for result in response["data"]["Get"]["ServicePublic"]]

  return "\n".join(page)

In [ ]:
def get_around_paragraph(url, paragrah_number, line_n, n_lines_around=3):
  url_filter = {
    "path": ["url"],
    "operator": "Equal",
    "valueText": url,
  }

  paragraph_filter = {
    "path": ["paragraph"],
    "operator": "Equal",
    "valueNumber": paragrah_number,
  }

  line_n_filter_gte = {
    "path": ["line_n"],
    "operator": "GreaterThanEqual",
    "valueNumber": line_n - n_lines_around,
  }

  line_n_filter_lte = {
    "path": ["line_n"],
    "operator": "LessThanEqual",
    "valueNumber": line_n + n_lines_around,
  }

  total_filter = {
      "operator": "And",
      "operands": [url_filter, paragraph_filter, line_n_filter_gte, line_n_filter_lte]
  }

  sort_order = [
    {
      'path': ['line_n'],
      'order': 'asc'
    },
  ]

  response = (
    client.query
    .get("ServicePublic", ["text", "url"])
    .with_where(total_filter)
    .with_sort(sort_order)
    .do()
  )

  page = [result['text'] for result in response["data"]["Get"]["ServicePublic"]]

  return "\n".join(page)

In [ ]:
def weaviate_connect(weaviate_endpoint, weaviate_api_key):
  # For using WCS
  import weaviate
  import json
  import os

  client = weaviate.Client(
      url = weaviate_endpoint,  # Replace with your endpoint
      auth_client_secret=weaviate.AuthApiKey(api_key=weaviate_api_key),  # Replace w/ your Weaviate instance API key
  )

  return client

In [ ]:
def get_prompt(question, results, context_lines=3):
  text = f"""A partir du contenu de ces textes répondez aux questions en fin de document.
  La réponse devra être la plus claire et détaillée possible et se conformer à cette convention de nommage:

  selon service-public.fr... <b>[numéro du document]</b>

  """

  for i, result in enumerate(results, start=1):
    title = result["title"]
    url = result["url"]
    paragraph = get_around_paragraph(result["url"], result["paragraph"], result["line_n"], context_lines)

    text += f"""
    [{i}]: ({title})[{url}]:
    {paragraph}

    """

  text += f"Question: {question}"

  return text


In [ ]:
def ask_gouvx(question, model, n_results=1):
  response = get_near_text(question, model)

  results = response["data"]["Get"]["ServicePublic"][:n_results]

  prompt = get_prompt(question, results)
  full_response = ask_llm(prompt)

  print("\nSources:")
  for i, result in enumerate(results, start=1):
    print(f"{[i]} ({result['title']})[{result['url']}]")

  return prompt, results, full_response

### Inference

In [ ]:
!pip install transformers
!pip install sentence_transformers
!pip install -U weaviate-client==3.22.1
!pip install openai

from sentence_transformers import SentenceTransformer
import openai

In [ ]:
weaviate_endpoint = ""
weaviate_api_key = ""
openai_api_key = ""

In [ ]:
client = weaviate_connect(weaviate_endpoint, weaviate_api_key)
openai.api_key = openai_api_key
model = SentenceTransformer("dangvantuan/sentence-camembert-large")

In [ ]:
question = "Ma voiture a un crit'air vraient très haut, c'est grave ?"
prompt, documents, reponse = ask_gouvx(question, model, 3)

### Eval

In [ ]:
print(''.join(reponse).replace(". ", "\n"))

Selon le texte de service-public.fr [1], quitter son travail pour déménager avec son partenaire peut être considéré comme un motif légitime de démission
Dans ce cas, vous pourriez être éligible à l'allocation chômage (ARE)
Cependant, il y a certaines conditions à remplir pour que votre démission soit considérée comme légitime par Pôle emploi.

Tout d'abord, votre partenaire doit déménager pour des raisons professionnelles, que ce soit une activité salariée ou non
De plus, votre nouvelle adresse ne doit pas vous permettre de continuer votre activité professionnelle
Il est également important de noter que vous devrez attendre 121 jours, soit environ 4 mois, avant de pouvoir demander à Pôle emploi de réexaminer votre dossier.

Il est recommandé de se référer au texte officiel pour obtenir tous les détails et les informations précises concernant l'éligibilité à l'allocation chômage en cas de démission pour déménagement avec son partenaire.


In [ ]:
eval_prompt = f"""Sur une échelle de un à dix, quel est la pertinence de cette réponse selon les documents proposés selon les critères suivants:
1. lien avec les documents
2. pertinence de la réponse à la question

Documents: {documents}

Question: {question}

Réponse: {reponse}

Répondez comme suit: 1: score, 2: score. Pas de texte supplémentaire"""

_ = ask_llm(eval_prompt)

1. lien avec les documents: 10
2. pertinence de la réponse à la question: 10

In [ ]:
print(prompt)

A partir du contenu de ces textes répondez aux questions en fin de document.
  La réponse devra être la plus claire et détaillée possible et se conformer à cette convention de nommage:

  selon service-public.fr... <b>[numéro du document]</b>

  
    [1]: (Un salarié peut-il toucher l'allocation chômage en cas de démission ? | Service-public.fr)[https://www.service-public.fr/particuliers/vosdroits/F89]:
    Textes de loi et références Code du travail : articles L5422-1 à L5422-2-1 Conditions d'attribution de l'allocation chômage Code du travail : article L5426-1-2 Dispositions particulières applicables aux bénéficiaires de l'allocation d'assurance à la suite d'une démission Code du travail : articles R5422-1 à R5422-2-3 Projet de reconversion professionnelle Convention 2017 et accords d'application relatifs à l'indemnisation du chômage Accords d'application n°12-§1er (réexamen des droits) et n°14 (démission légitime)Décret n° 2019-797 du 26 juillet 2019 relatif au régime d'assurance ch